In [5]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import os
import requests
import pandas as pd
import mariadb
import sys

client_id = 'e8730ef24c72408dac82416586828b8f'
client_secret = 'acd0e8fe228a4d7287132bd6e6312094'
redirect_Uri = 'http://localhost:8080/callback'
            
uri = 'spotify:user:e8730ef24c72408dac82416586828b8f'

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id, client_secret=client_secret, redirect_uri=redirect_Uri))

results = sp.current_user()
#print(results["display_name"])
print(results)

playlists = []

results = sp.current_user_playlists()
print(results)





{'display_name': 'sushamacardozo', 'external_urls': {'spotify': 'https://open.spotify.com/user/31kwtwm2w4zyqzbmw4ve7smdtuhm'}, 'followers': {'href': None, 'total': 0}, 'href': 'https://api.spotify.com/v1/users/31kwtwm2w4zyqzbmw4ve7smdtuhm', 'id': '31kwtwm2w4zyqzbmw4ve7smdtuhm', 'images': [], 'type': 'user', 'uri': 'spotify:user:31kwtwm2w4zyqzbmw4ve7smdtuhm'}
{'href': 'https://api.spotify.com/v1/users/31kwtwm2w4zyqzbmw4ve7smdtuhm/playlists?offset=0&limit=50', 'items': [{'collaborative': False, 'description': '', 'external_urls': {'spotify': 'https://open.spotify.com/playlist/4JuNUx4pKJhUC7J1JBQkur'}, 'href': 'https://api.spotify.com/v1/playlists/4JuNUx4pKJhUC7J1JBQkur', 'id': '4JuNUx4pKJhUC7J1JBQkur', 'images': [], 'name': 'Hindi Songs', 'owner': {'display_name': 'sushamacardozo', 'external_urls': {'spotify': 'https://open.spotify.com/user/31kwtwm2w4zyqzbmw4ve7smdtuhm'}, 'href': 'https://api.spotify.com/v1/users/31kwtwm2w4zyqzbmw4ve7smdtuhm', 'id': '31kwtwm2w4zyqzbmw4ve7smdtuhm', 'type'

In [ ]:
total_playlists = results['total']

for i in range(0,total_playlists):
    playlists.append(results['items'][i])

print(playlists)

In [ ]:
columns = []
row_list = []

for i in range(0,total_playlists):
    #print(pl.keys())
    rows = []
    for key, value in playlists[i].items():
        datatype = str(type(value))[8:len(str(type(value)))-2]
        if value == None:
            value = ""
        #print(key, datatype)
        if i == 0:
            if key == "external_urls":
                columns.append("pl_" + key + " VARCHAR(255)")
            elif key == "images":
                columns.append("pl_" + key + " TEXT")
            elif key == "owner":
                columns.append("pl_" + key + " VARCHAR(255)")
            elif key == "primary_color":
                columns.append("pl_" + key + " VARCHAR(255)")
            elif key == "tracks":
                columns.append("pl_no_of_tracks VARCHAR(255)")
            else:
                if datatype == "str":
                    columns.append("pl_" + key + " VARCHAR(255)")
                elif datatype == "int":
                    columns.append("pl_" + key + " INT")
                elif datatype == "bool":
                    columns.append("pl_" + key + " BOOL")
            #rows.append(" VALUES ")
        if key == "external_urls":
            rows.append(str(value))
        elif key == "images":
            rows.append(str(value))
        elif key == "owner":
            rows.append(value["id"])
        elif key == "primary_color":
            rows.append(value)
        elif key == "tracks":
            rows.append(value["total"])
        else:
            rows.append(value)
    #print(tuple(rows))
    row_list.append(tuple(rows))
#print(tuple(columns)) 
column_list = ""
for c in columns:
    column_list += c + ", "
column_list = "(" + column_list[:-2] + ")"
print(column_list)
print(row_list)   
#print(list)

In [ ]:
try:
    conn = mariadb.connect(
        user="root",
        password="password",
        host="localhost",
        port=3306,
        database="test_db"
    )
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")
    sys.exit(1)
conn.autocommit = False
print("Connected Successfully!")

In [ ]:
try:
    cur = conn.cursor()
    create_table = f"CREATE TABLE IF NOT EXISTS playlists {column_list} "
    #print(create_table)
    cur.execute("DROP TABLE IF EXISTS playlist")
    cur.execute(create_table)

except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")
    sys.exit(1)
    
print("Table created Successfully!")


In [ ]:
try:
    cur.execute("SELECT * FROM playlists")
    delete_rows = "DELETE FROM playlists"
    cur.execute(delete_rows)
    print("O Rows in playlists Table!")
    for r in row_list:
        insert_values = f"INSERT INTO playlists VALUES {r}" 
        #print(insert_values)
        cur.execute(insert_values)
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")
    sys.exit(1)

conn.commit()
print("Rows inserted Successfully!")
cur.execute("SELECT * FROM playlists")
print(cur.fetchall())


In [ ]:
conn.close()